In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn.model_selection import KFold, cross_val_predict
from scipy.sparse import hstack, csr_matrix
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

In [2]:
path = 'data/'

In [3]:
%%time

train_unique = pd.read_parquet(path+'train_unique.parquet')
receipt_words_total = pd.read_parquet(path+'receipt_words_total.parquet')

CPU times: user 3.62 s, sys: 1.02 s, total: 4.64 s
Wall time: 2.94 s


In [4]:
mask = (receipt_words_total['count_y'] > 100) & (receipt_words_total['perc_xy'] < 0.999)
receipt_words_total_new = receipt_words_total[mask]

df_dict_50words = receipt_words_total_new.groupby(['word_x'])['word_y'].apply(lambda x: list(x[:150]))
dict_50words = dict(df_dict_50words)

В словаре dict_50words для каждого слова лежит лист "близких" к нему слов. 

In [5]:
dict_50words['соррентин']

['sorrento',
 'частичн',
 'частич',
 'выпеч',
 'моцарелл',
 'ит',
 'gusto',
 'замороз',
 'моцар',
 'си',
 'импорт',
 'пицц',
 'ветчин',
 'ч',
 'мо',
 'грамм',
 'порц']

In [6]:
def create_discr(s, top_=10):
    arr = s.split()
    arr_total = []
    for s_i in arr:
        arr_total += dict_50words.get(s_i, [])[:top_]
    return ' '.join(np.unique(arr_total))

Теперь для каждого товара сделаем "ассоциативное описание" из набор близких слов.

In [10]:
train_unique.sample(10)

,item_name,category_id,item_name_clean
3382,"0,5Л ПИВО ЧЕРНОВАР ТЁМН Ж/Б",0,литр пив черновар темн б
13814,"Городская булочка в/с 0,2кг Сочихлеб(шт)",84,городск булочк кг сочихлеб штук
19022,"Клей ""Супермомент"" (3г)",114,кле супермомент грамм
11962,Вафли с шоколадом 90/50/50,84,вафл шоколад
41028,ФЛОМАСТЕРЫ ФЛОМАСТЕРЫ 12ЦВ BRUNOVISCONTI HAPPY...,31,фломастер фломастер цв brunovisconti happycolor
28699,ПИРОЖКИ С КАРТОФЕЛЕМ,84,пирожок картофел
32577,РЕЗИНКА ПОДВЕС.ГЛУШ.01 задняя,4,резинк подвес глуш задн
30760,"Пиво светлое Гёссер алк.4,7% 0,45л ж/б",0,пив светл гессер алк процент литр б
32327,Пряник Солнечный 1кг апельсин Русские лакомства,84,пряник солнечн кг апельсин русск лакомств
6553,A19ADEJUW02-99 XS Куртка для бега женская Wom...,62,a adejuw x куртк бег женск woman s running jac...


In [11]:
s = 'литр пив черновар темн б'
create_discr(s, 5)

'draught fitnes tr zat алкоторг бочк бэт велкопоповицкийкозел лаймонфреш лаошан оболочкойвсян оттингер пастерилиз пив сек сероглазок сихот тда темн хамовн хеллс царингер чех шварцбир'

На деле полезнее брать не 5, а 65 ближайших слов для каждого слова из названия.

In [12]:
train_unique['random_words'] = train_unique['item_name_clean'].apply(lambda s: create_discr(s, top_=65))
train_unique['random_words'] = train_unique['random_words'].fillna('')

In [13]:
N_JOBS = 8
y = train_unique['category_id']
y_unique = np.unique(y)
folds = KFold(8, shuffle=True, random_state=0)

In [14]:
%%time

tfidf1 = TfidfVectorizer(max_features=60000, ngram_range=(1, 1))
tfidf2 = TfidfVectorizer(max_features=60000, ngram_range=(1, 2))
tfidf3 = TfidfVectorizer(max_features=75000, ngram_range=(3, 5), analyzer="char_wb")

X_name = tfidf1.fit_transform(train_unique['item_name'])
X_random = tfidf2.fit_transform(train_unique['random_words'])
X_char = tfidf3.fit_transform(train_unique['item_name'])

CPU times: user 42.6 s, sys: 1.37 s, total: 43.9 s
Wall time: 44.1 s


In [15]:
X = csr_matrix(hstack([X_name, X_random, X_char]))

In [17]:
%%time

clf = LinearSVC(random_state=1, C=0.4)
predicts = cross_val_predict(clf, X, y, cv=folds, n_jobs=N_JOBS, method='predict')
score = f1_score(y, predicts, average='weighted')
print(f'{score:.4f}')

0.8591
CPU times: user 210 ms, sys: 557 ms, total: 767 ms
Wall time: 5min 2s


На lb получается примерно 0.86. С помощью доразметки можно еще немного поднять.